<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/rge_validator_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# rge/validator.py
from __future__ import annotations
from dataclasses import dataclass, asdict
from typing import Callable, Iterable, Optional, Dict, Any, List, Tuple, Protocol, Union
import numpy as np
import re
import json
import math
import time

Array = np.ndarray

class SimOutput(Protocol):
    t: Array            # shape (T,)
    y: Array            # shape (S, T)
    states: Iterable[str]  # e.g., ['x', 'y']

Token = str

def _to_tokens(eq: Any) -> List[Token]:
    # SymPy-safe: fall back to string if no .args
    s = str(eq)
    # Simple tokenizer: identifiers, numbers, operators
    return [tok for tok in re.findall(r"[A-Za-z_]+|\d+\.\d+|\d+|[+\-*/^=()]", s) if tok.strip()]

def _description_length(equations: Iterable[Any]) -> int:
    # Weighted token count for MDL-like measure
    weights = {
        "+": 1, "-": 1, "*": 1, "/": 2, "^": 2, "=": 0,
        "sin": 3, "cos": 3, "exp": 3, "log": 3
    }
    total = 0
    for eq in equations:
        for tok in _to_tokens(eq):
            total += weights.get(tok, 1)  # default weight
    return total

def _simplicity_score(desc_len: int, lam: float = 0.03) -> float:
    # Smoothly decreasing in description length, bounded in (0,1]
    return float(np.exp(-lam * max(desc_len, 0)))

def _masked_var(y_true: Array, axis=None) -> float:
    mask = ~np.isnan(y_true)
    if not np.any(mask):
        return 0.0
    centered = y_true[mask] - np.nanmean(y_true)
    v = float(np.mean(centered**2))
    return v

def _accuracy_with_gt(y_pred: Array, y_true: Array) -> Tuple[float, Dict[str, Any]]:
    if y_pred.shape != y_true.shape:
        raise ValueError(f"Shape mismatch: pred {y_pred.shape} vs gt {y_true.shape}")
    mask = ~np.isnan(y_true)
    if not np.any(mask):
        return 0.0, {"mse": None, "var": None, "masked_fraction": 0.0}

    diff = (y_pred - y_true)[mask]
    mse = float(np.mean(diff**2))
    var = _masked_var(y_true)
    # Variance-normalized error; fall back if var≈0
    if var > 0:
        err = mse / var
    else:
        # Scale by squared amplitude to avoid division by ~0
        amp = float(np.nanmax(np.abs(y_true)) or 1.0)
        err = mse / (amp**2 if amp > 0 else 1.0)
    acc = float(np.exp(-err))
    return acc, {"mse": mse, "var": var, "norm_err": err}

def _finite_diff(y: Array, t: Array) -> Array:
    # Central differences in time; shape (S, T)
    dt = np.diff(t)
    if np.any(dt <= 0):
        raise ValueError("Time vector must be strictly increasing")
    S, T = y.shape
    dy = np.empty_like(y)
    # Forward/backward at ends
    dy[:, 0] = (y[:, 1] - y[:, 0]) / dt[0]
    dy[:, -1] = (y[:, -1] - y[:, -2]) / dt[-1]
    # Central for interior
    mid_dt = (t[2:] - t[:-2]) / 2.0
    dy[:, 1:-1] = (y[:, 2:] - y[:, :-2]) / mid_dt
    return dy

def _smoothness_score(y: Array, t: Array) -> float:
    # Penalize high curvature relative to scale
    dy = _finite_diff(y, t)
    d2y = _finite_diff(dy, t)
    num = np.nanmean(d2y**2)
    denom = (np.nanmean(y**2) or 1.0)
    ratio = float(num / denom)
    return float(np.exp(-0.1 * ratio))

def _reversibility_score(y: Array, t: Array) -> float:
    # Time-reversal invariance proxy: compare y with reversed time interpolation
    # Simple proxy: correlation between forward and reversed sequences
    y_f = (y - np.nanmean(y, axis=1, keepdims=True)) / (np.nanstd(y, axis=1, keepdims=True) + 1e-12)
    y_r = np.flip(y_f, axis=1)
    # Mean absolute correlation across states
    corr = np.mean([np.abs(np.corrcoef(y_f[i], y_r[i])[0,1]) for i in range(y.shape[0]) if y.shape[1] > 3])
    if np.isnan(corr):
        corr = 0.0
    return float(0.5 + 0.5 * corr)  # map [-1,1]→[0,1]

def _energy_drift_score(
    y: Array, t: Array, energy_fn: Optional[Callable[[Array], Array]] = None
) -> Tuple[float, Optional[float]]:
    if energy_fn is None:
        return 0.5, None  # neutral if no energy provided
    E = energy_fn(y)  # expects shape (T,)
    if E is None or np.all(np.isnan(E)):
        return 0.5, None
    # Drift relative to median energy
    drift = float(np.nanmax(np.abs(E - np.nanmedian(E))) / (np.nanmedian(np.abs(E)) + 1e-9))
    score = float(np.exp(-0.2 * drift))
    return score, drift

def _kl_divergence(p: Array, q: Array) -> float:
    eps = 1e-12
    p = p + eps
    q = q + eps
    p = p / p.sum()
    q = q / q.sum()
    return float(np.sum(p * np.log(p / q)))

def _token_hist(tokens: List[Token], vocab: List[Token]) -> Array:
    idx = {tok: i for i, tok in enumerate(vocab)}
    counts = np.zeros(len(vocab), dtype=float)
    for t in tokens:
        if t in idx:
            counts[idx[t]] += 1
    if counts.sum() == 0:
        counts += 1.0  # uniform if empty
    return counts

def _novelty_score(
    equations: Iterable[Any],
    prior_vocab: List[Token],
    prior_freq: Array,
    beta: float = 0.6
) -> Tuple[float, Dict[str, Any]]:
    # Build empirical distribution from equations
    toks: List[Token] = []
    for eq in equations:
        toks.extend(_to_tokens(eq))
    p_hist = _token_hist(toks, prior_vocab)
    kl = _kl_divergence(p_hist, prior_freq)
    # Map KL (0..∞) to (0,1]: higher KL ⇒ higher novelty
    score = float(1.0 - np.exp(-beta * kl))
    return score, {"kl": kl, "token_count": int(p_hist.sum())}

@dataclass
class ScoreReport:
    total: float
    passed: bool
    components: Dict[str, float]
    details: Dict[str, Any]
    threshold: float
    version: str = "meta-validator/1.0.0"
    timestamp: float = time.time()

    def to_json(self) -> str:
        return json.dumps({
            **asdict(self),
        }, indent=2, sort_keys=True)

class MetaValidator:
    def __init__(
        self,
        threshold: float = 0.75,
        alpha: Tuple[float, float, float] = (0.5, 0.3, 0.2),
        prior_vocab: Optional[List[Token]] = None,
        prior_freq: Optional[Array] = None,
        seed: Optional[int] = 42,
    ):
        self.threshold = float(threshold)
        self.w_acc, self.w_simp, self.w_nov = alpha
        self.rng = np.random.default_rng(seed)
        # Default prior: minimal symbolic vocabulary
        default_vocab = ["x","y","z","t","+","-","*","/","^","=","sin","cos","exp","log","1","2","0"]
        self.prior_vocab = prior_vocab or default_vocab
        if prior_freq is None:
            # Smooth/heavy-tailed prior
            self.prior_freq = np.array([3,3,1,2,5,4,5,3,2,0,1,1,1,1,2,1,2], dtype=float)
        else:
            self.prior_freq = np.array(prior_freq, dtype=float)
        if len(self.prior_vocab) != len(self.prior_freq):
            raise ValueError("prior_vocab and prior_freq must align")

    def score(
        self,
        theory: Dict[str, Any],
        sim_output: SimOutput,
        ground_truth: Optional[Array] = None,
        energy_fn: Optional[Callable[[Array], Array]] = None,
    ) -> ScoreReport:
        """
        Compute a composite score:
        S = w_acc * acc + w_simp * simp + w_nov * nov

        Args:
          theory: dict with at least {"equations": Iterable[Any]}
          sim_output: provides .t (T,), .y (S,T), .states (Iterable[str])
          ground_truth: optional array shaped like sim_output.y
          energy_fn: optional function mapping y (S,T) -> energy (T,)
        """
        # Validate sim_output
        t = np.asarray(sim_output.t)
        y = np.asarray(sim_output.y)
        if y.ndim != 2:
            raise ValueError(f"sim_output.y must be 2D (S,T), got {y.shape}")
        if t.ndim != 1 or t.shape[0] != y.shape[1]:
            raise ValueError("Time vector length must match y's time dimension")
        equations = theory.get("equations", [])
        if not isinstance(equations, (list, tuple)) or len(equations) == 0:
            raise ValueError("theory['equations'] must be a non-empty list")

        # 1) Accuracy
        if ground_truth is not None:
            acc, acc_det = _accuracy_with_gt(y, np.asarray(ground_truth))
        else:
            # Proxy blend
            rev = _reversibility_score(y, t)
            smo = _smoothness_score(y, t)
            eng, drift = _energy_drift_score(y, t, energy_fn)
            acc = float(0.5 * rev + 0.3 * smo + 0.2 * eng)
            acc_det = {"proxy": True, "reversibility": rev, "smoothness": smo, "energy": eng, "energy_drift": drift}

        # 2) Simplicity
        desc_len = _description_length(equations)
        simp = _simplicity_score(desc_len)
        simp_det = {"description_length": desc_len}

        # 3) Novelty
        nov, nov_det = _novelty_score(equations, self.prior_vocab, self.prior_freq)

        total = float(self.w_acc * acc + self.w_simp * simp + self.w_nov * nov)
        passed = bool(total >= self.threshold)

        details = {
            "accuracy": acc_det,
            "simplicity": simp_det,
            "novelty": nov_det,
            "weights": {"acc": self.w_acc, "simp": self.w_simp, "nov": self.w_nov},
        }
        return ScoreReport(
            total=total,
            passed=passed,
            components={"accuracy": acc, "simplicity": simp, "novelty": nov},
            details=details,
            threshold=self.threshold,
        )

    def calibrate_weights(
        self,
        labeled_samples: List[Dict[str, Any]],
        method: str = "grid",
    ) -> Tuple[Tuple[float,float,float], Dict[str, Any]]:
        """
        Fit alpha weights to best separate accepted vs rejected samples.
        labeled_samples: list of {"components": {"accuracy":a,"simplicity":s,"novelty":n}, "label": {0,1}}
        Returns (w_acc, w_simp, w_nov), diagnostics
        """
        comps = np.array([[d["components"]["accuracy"], d["components"]["simplicity"], d["components"]["novelty"]] for d in labeled_samples])
        labels = np.array([d["label"] for d in labeled_samples], dtype=int)
        if method == "grid":
            grid = np.linspace(0.0, 1.0, 11)
            best_w, best_score = (0.5,0.3,0.2), -1.0
            for wa in grid:
                for ws in grid:
                    wn = 1.0 - wa - ws
                    if wn < 0 or wn > 1:
                        continue
                    scores = comps @ np.array([wa, ws, wn])
                    preds = (scores >= self.threshold).astype(int)
                    f1 = self._f1(labels, preds)
                    if f1 > best_score:
                        best_score = f1
                        best_w = (float(wa), float(ws), float(wn))
            self.w_acc, self.w_simp, self.w_nov = best_w
            return best_w, {"f1": best_score, "method": method}
        else:
            raise NotImplementedError("Only 'grid' method is provided for now.")

    @staticmethod
    def _f1(y_true: Array, y_pred: Array) -> float:
        tp = np.sum((y_true==1) & (y_pred==1))
        fp = np.sum((y_true==0) & (y_pred==1))
        fn = np.sum((y_true==1) & (y_pred==0))
        prec = tp / (tp + fp + 1e-12)
        rec  = tp / (tp + fn + 1e-12)
        if prec + rec == 0:
            return 0.0
        return 2 * prec * rec / (prec + rec)